# Goodreads - Web scraper



In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

/var/folders/lr/8c5hs04n5c16pwk_y0fr6fvr0000gn/T/ipykernel_47983/3972107164.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/Users/manasibhagwat/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
book_titles = []
authors = []
avg_ratings = []
ratings = []
published_years = []
count =[]

In [34]:
user_agent = "webscraper"
pages_to_scrape = 500
request_delay = 3

# Going through each page to extract data
for page in range(1, pages_to_scrape + 1):
    
    url = "https://www.goodreads.com/search?page=" + str(page) + "&q=self+help&qid=ozFSiGJBE0"
   
    try:
        # HTTP GET request to the URL
        headers = {"User-Agent": user_agent}
        response = requests.get(url, headers=headers).text
        soup = BeautifulSoup(response, "html5lib")
    
        if soup.title and "service unavailable" in soup.title.text.lower():
            continue

        # table with all books
        table = soup.find_all("tbody")[0]

        # loop through all rows of table books
        for row in table.find_all("tr"):
            cells = row.find_all("td")[1]

            title = cells.find("a").find("span").text
            book_titles.append(title)
            

            author = cells.find("a", class_="authorName").text
            authors.append(author)

            all_ratings = cells.find_all('span', class_ = 'minirating')
            all_ratings_text = all_ratings[0].text.strip()
            pattern_2 = re.compile(r"(\d\.?\d*)\savg")
            avg_ratings.append(pattern_2.search(all_ratings_text).group(1))

            pattern_4 = re.compile(r"(\d\,?\d*) rating")
            ratings_matches = pattern_4.search(all_ratings_text)
            ratings.append(ratings_matches.group(1) if ratings_matches else 0)  

            year_info = cells.find("span", class_="greyText smallText uitext").text.split()
            year = None
            for item in year_info:
                if item.isdigit() and len(item) == 4:
                    year = item
                    break
            if year:
                published_years.append(year)
            else:
                published_years.append(0)  

        # Sleep to add a delay between requests
        time.sleep(request_delay)
    
    except requests.exceptions.RequestException as e:
        print(f"Error on page {page}: {e}")

    except IndexError as e:
        print(f"Index error on page {page}: {e}")
        continue

    except Exception as e:
        print(f"Unexpected error on page {page}: {e}")


Index error on page 38: list index out of range
Index error on page 40: list index out of range
Index error on page 117: list index out of range
Index error on page 439: list index out of range


In [35]:

data = {
    "Title": book_titles,
    "Author": authors,
    "Average Rating": avg_ratings,
    "Rating": ratings,
    "Year Published": published_years
}

In [36]:
goodreads = pd.DataFrame(data)
goodreads.head()

,Title,Author,Average Rating,Rating,Year Published
0,Self-Help,Lorrie Moore,4.14,"3,786",1985
1,What's Our Problem?: A Self-Help Book for Soci...,Tim Urban,4.28,"3,397",2023
2,Parenting from the Inside Out: How a Deeper Se...,Daniel J. Siegel,4.16,"4,799",2003
3,How to Be Fine: What We Learned from Living by...,Jolenta Greenberg,3.54,"4,414",2020
4,Get Out of Your Mind and Into Your Life: The N...,Steven C. Hayes,4.06,"3,056",2005


In [37]:
len(goodreads)

6230

In [38]:
goodreads.to_csv("goodreads_data.csv", index=False)